# Diamond Price Prediction Web Application

## Step 1: Loading Dependencies

In this section, we import the required libraries for building the web application. dash is the main library for creating interactive web applications, dcc provides core components for Dash, and html provides HTML components. We also import libraries for loading our pre-trained model (keras) and the PCA model (joblib), as well as standard data manipulation libraries (numpy and pandas).

In [1]:
# Import necessary libraries
import dash
import dash_core_components as dcc
from dash import html
from dash.dependencies import Input, Output
from dash.dependencies import Input, Output, State
from keras.models import load_model
import joblib

import numpy as np
import pandas as pd


C:\Users\Sesan\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  This is separate from the ipykernel package so we can avoid doing imports until


# Section 2: Load Pre-Trained Model and PCA

Here, we load the pre-trained neural network model (diamond-neural-network.h5) and the PCA model (pca.joblib) that were provided.

In [2]:
# Load the pre-trained model and PCA
model = load_model('diamond-neural-network.h5')
pca = joblib.load('pca.joblib')

C:\Users\Sesan\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator PCA from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


# Section 3: Initialize the Dash App

In this section, we initialize the Dash app. We also include external CSS styling to enhance the visual appearance of the web application.

In [3]:
# Initialize the Dash app
app = dash.Dash(__name__)
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})


# Section 4: Define Input Components for Diamond Characteristics

This section defines the input components for various diamond characteristics. Numeric input fields are used for carat, depth, table, and dimensions (x, y, z), while dropdowns are used for cut, color, and clarity. Default values are provided for each input.



In [4]:
# Define input components for various diamond characteristics
input_carat = dcc.Input(id='carat', type='number', placeholder='Carat', value=0.6)
input_depth = dcc.Input(id='depth', type='number', placeholder='Depth', value=20)
input_table = dcc.Input(id='table', type='number', placeholder='Table', value=40)
input_x = dcc.Input(id='x', type='number', placeholder='x value', value=5)
input_y = dcc.Input(id='y', type='number', placeholder='y value', value=6)
input_z = dcc.Input(id='z', type='number', placeholder='z value', value=7)

cut_values = ['Fair', 'Good', 'Ideal', 'Premium', 'Very Good']
cut_options = [{'label': x, 'value': x} for x in cut_values]
input_cut = dcc.Dropdown(id='cut', options=cut_options, value='Ideal')

color_values = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
color_options = [{'label': x, 'value': x} for x in color_values]
input_color = dcc.Dropdown(id='color', options=color_options, value='G')

clarity_values = ['I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2']
clarity_options = [{'label': x, 'value': x} for x in clarity_values]
input_clarity = dcc.Dropdown(id='clarity', options=clarity_options, value='SI1')


# Section 5: Layout of the App

In this section, we define the overall layout of the web application using Dash's HTML components. The layout includes headers, input components organized in a grid structure, and an empty space for the output.

In [5]:
# Layout of the app
app.layout = html.Div([
    html.H1('IDR Predict Diamond Prices'),
    html.H2('Enter Diamond Characteristics to Get Predicted Price'),
    html.Div([
        html.Div([html.H3('Carat:'), input_carat], className="four columns"),
        html.Div([html.H3('Depth:'), input_depth], className="four columns"),
        html.Div([html.H3('Table:'), input_table], className="four columns"),
        html.Div([html.H3('x value:'), input_x], className="four columns"),
        html.Div([html.H3('y value:'), input_y], className="four columns"),
        html.Div([html.H3('z value:'), input_z], className="four columns"),
        html.Div([html.H3('Cut:'), input_cut], className="four columns"),
        html.Div([html.H3('Color:'), input_color], className="four columns"),
        html.Div([html.H3('Clarity:'), input_clarity], className="four columns")
    ], className="row"),
    # Add a "Predict" button
    html.Button('Predict', id='predict-button', n_clicks=0, style={'margin-top': '10px'}),
    
    
    html.H1(id='output', style={'margin-top': '50px', 'text-align': 'center'})
])



# Section 6: Callback Function for Prediction

Here, we define the function get_prediction that takes user inputs, processes them, and returns the predicted price. This section includes a placeholder for the existing code responsible for predictions and any adjustments needed.

In [6]:
# Function to get predictions
def get_prediction(carat, depth, table, x, y, z, cut, color, clarity):
    # Define the columns for the input DataFrame
    cols = ['carat', 'depth', 'table',
            'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good',
            'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J',
            'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2',
            'dim_index']

    # Create dictionaries to map categorical values to one-hot encoded columns
    cut_dict = {x: 'cut_' + x for x in cut_values[1:]}
    color_dict = {x: 'color_' + x for x in color_values[1:]}
    clarity_dict = {x: 'clarity_' + x for x in clarity_values[1:]}

    # Create a DataFrame with a single row of zeros
    df = pd.DataFrame(data=np.zeros((1, len(cols))), columns=cols)

    # Assign numeric characteristics
    df.loc[0, 'carat'] = carat
    df.loc[0, 'depth'] = depth
    df.loc[0, 'table'] = table

    # Transform dimensions into a single dim_index using PCA
    dims_df = pd.DataFrame(data=[[x, y, z]], columns=['x', 'y', 'z'])
    df.loc[0, 'dim_index'] = pca.transform(dims_df).flatten()[0]

    # Use one-hot encoding for categorical features
    if cut != 'Fair':
        df.loc[0, cut_dict[cut]] = 1

    if color != 'D':
        df.loc[0, color_dict[color]] = 1

    if clarity != 'I1':
        df.loc[0, clarity_dict[clarity]] = 1

    # TODO: Uncomment the following line if scaler is used and available
    # df.loc[:, numerical_features] = scaler.transform(df.loc[:, numerical_features])

    # Get predictions using the trained neural network
    prediction = model.predict(df.values).flatten()[0]

    # Check for extreme predictions and cap them at a predefined threshold (adjust as needed)
    if prediction > 20.0:
        prediction = 20.0

    # Transform the log-prices to prices
    prediction = np.exp(prediction)

    # Return the predicted price as an integer
    return int(prediction)


# Section 8: Run the App

The final section runs the Dash app if the script is executed directly.

In [7]:
# Define callback for prediction updates

# Callback to update the output based on user input
predictors = ['carat', 'depth', 'table', 'x', 'y', 'z', 'cut', 'color', 'clarity']
@app.callback(Output('output', 'children'), [Input(x, 'value') for x in predictors])
def show_prediction(carat, depth, table, x, y, z, cut, color, clarity):
    pred = get_prediction(carat, depth, table, x, y, z, cut, color, clarity)
    return str("Predicted Price: {:,}".format(pred))

In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\Sesan\anaconda3\envs\PythonData\lib\site-packages\dash\resources.py:63: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



1/1 [==============================] - ETA: 0s

C:\Users\Sesan\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:444: UserWarning:

X has feature names, but PCA was fitted without feature names



1/1 [==============================] - 0s 115ms/step
